In [34]:
import pandas as pd
import holidays
import numpy as np
from holidays import Germany

Read in .csv file.

In [35]:
df = pd.read_csv('/workspaces/Room_7_Bakery_Prediction/0_DataPreparation/Data/complete_dataset.csv')

Convert to datetime.

In [36]:
df['date'] = pd.to_datetime(df['date'])
df.dtypes

id                            float64
date                   datetime64[ns]
Warengruppe                   float64
umsatz                        float64
KielerWoche                   float64
Bewoelkung                    float64
Temperatur                    float64
Windgeschwindigkeit           float64
Wettercode                    float64
dtype: object

Dataset information.

In [37]:
print(f"Columns: {df.columns.tolist()}")
print(f"Shape: {df.shape}")
print(f"Time period: {df['date'].min()} to {df['date'].max()}")
print(f"Number of rows: {len(df):,}")
print(f"Number of product groups: {df['Warengruppe'].nunique()}")
print("\nFirst rows:")
print(df.head(10))

Columns: ['id', 'date', 'Warengruppe', 'umsatz', 'KielerWoche', 'Bewoelkung', 'Temperatur', 'Windgeschwindigkeit', 'Wettercode']
Shape: (11164, 9)
Time period: 2013-07-01 00:00:00 to 2019-07-30 00:00:00
Number of rows: 11,164
Number of product groups: 6

First rows:
          id       date  Warengruppe      umsatz  KielerWoche  Bewoelkung  \
0  1307015.0 2013-07-01          5.0  317.475875          NaN         6.0   
1  1307014.0 2013-07-01          4.0   65.890169          NaN         6.0   
2  1307012.0 2013-07-01          2.0  535.856285          NaN         6.0   
3  1307013.0 2013-07-01          3.0  201.198426          NaN         6.0   
4  1307011.0 2013-07-01          1.0  148.828353          NaN         6.0   
5  1307021.0 2013-07-02          1.0  159.793757          NaN         3.0   
6  1307022.0 2013-07-02          2.0  546.780787          NaN         3.0   
7  1307023.0 2013-07-02          3.0  265.261254          NaN         3.0   
8  1307024.0 2013-07-02          4.0   7

# Add public holidays

In [38]:
# Public holidays for Schleswig-Holstein from 2013-2019
de_sh_holidays = holidays.Germany(prov='SH', years=range(2013, 2020))

# Feature: Is_Holiday
df['Is_Holiday'] = df['date'].apply(lambda x: 1 if x in de_sh_holidays else 0)

# Sort for correct lag features
df = df.sort_values(['date', 'Warengruppe']).reset_index(drop=True)

Adding additional features 'Day_Before_Holiday' and 'Day_After_Holiday'.

In [39]:
# Feature: Day before holiday (important for bakeries - higher sales!)
# Grouped by product group to avoid incorrect shifts across groups
df['Day_Before_Holiday'] = df.groupby('Warengruppe')['Is_Holiday'].shift(-1).fillna(0).astype(int)

# Feature: Day after holiday
df['Day_After_Holiday'] = df.groupby('Warengruppe')['Is_Holiday'].shift(1).fillna(0).astype(int)

print(f"   • Number of holidays: {df['Is_Holiday'].sum()} days")
print(f"   • Day before holiday: {df['Day_Before_Holiday'].sum()} days")
print(f"   • Day after holiday: {df['Day_After_Holiday'].sum()} days")


   • Number of holidays: 120 days
   • Day before holiday: 120 days
   • Day after holiday: 120 days


Adjust timeframe to our data.

In [40]:
# Which holidays are in the time period?
# Convert pandas Timestamps to date objects for comparison
min_date = df['date'].min().date()
max_date = df['date'].max().date()

holidays_in_period = sorted([date for date in de_sh_holidays 
                             if min_date <= date <= max_date])
print(f"\n   Holidays in period ({len(holidays_in_period)} days):")
for holiday in holidays_in_period[:10]:  # Show first 10
    print(f"   - {holiday.strftime('%d.%m.%Y')} ({de_sh_holidays.get(holiday)})")
if len(holidays_in_period) > 10:
    print(f"   ... and {len(holidays_in_period) - 10} more")


   Holidays in period (56 days):
   - 03.10.2013 (German Unity Day)
   - 25.12.2013 (Christmas Day)
   - 26.12.2013 (Second Day of Christmas)
   - 01.01.2014 (New Year's Day)
   - 18.04.2014 (Good Friday)
   - 21.04.2014 (Easter Monday)
   - 01.05.2014 (Labor Day)
   - 29.05.2014 (Ascension Day)
   - 09.06.2014 (Whit Monday)
   - 03.10.2014 (German Unity Day)
   ... and 46 more


# Add school vacations

In [41]:
# School vacation periods for Schleswig-Holstein (01.07.2013 - 30.07.2019)
vacation_data = [
    # 2013
    {'Start': '2013-07-01', 'End': '2013-08-03', 'Type': 'Summer'},
    {'Start': '2013-10-04', 'End': '2013-10-18', 'Type': 'Autumn'},
    {'Start': '2013-12-23', 'End': '2014-01-06', 'Type': 'Christmas'},
    
    # 2014
    {'Start': '2014-04-16', 'End': '2014-05-02', 'Type': 'Easter'},
    {'Start': '2014-05-30', 'End': '2014-05-30', 'Type': 'Whitsun'},
    {'Start': '2014-07-14', 'End': '2014-08-23', 'Type': 'Summer'},
    {'Start': '2014-10-13', 'End': '2014-10-25', 'Type': 'Autumn'},
    {'Start': '2014-12-22', 'End': '2015-01-06', 'Type': 'Christmas'},
    
    # 2015
    {'Start': '2015-04-01', 'End': '2015-04-17', 'Type': 'Easter'},
    {'Start': '2015-05-15', 'End': '2015-05-15', 'Type': 'Whitsun'},
    {'Start': '2015-07-20', 'End': '2015-08-29', 'Type': 'Summer'},
    {'Start': '2015-10-19', 'End': '2015-10-31', 'Type': 'Autumn'},
    {'Start': '2015-12-21', 'End': '2016-01-06', 'Type': 'Christmas'},
    
    # 2016
    {'Start': '2016-03-24', 'End': '2016-04-09', 'Type': 'Easter'},
    {'Start': '2016-05-06', 'End': '2016-05-06', 'Type': 'Whitsun'},
    {'Start': '2016-07-25', 'End': '2016-09-03', 'Type': 'Summer'},
    {'Start': '2016-10-17', 'End': '2016-10-29', 'Type': 'Autumn'},
    {'Start': '2016-12-23', 'End': '2017-01-06', 'Type': 'Christmas'},
    
    # 2017
    {'Start': '2017-04-07', 'End': '2017-04-21', 'Type': 'Easter'},
    {'Start': '2017-05-26', 'End': '2017-05-26', 'Type': 'Whitsun'},
    {'Start': '2017-07-24', 'End': '2017-09-02', 'Type': 'Summer'},
    {'Start': '2017-10-16', 'End': '2017-10-27', 'Type': 'Autumn'},
    {'Start': '2017-12-21', 'End': '2018-01-06', 'Type': 'Christmas'},
    
    # 2018
    {'Start': '2018-03-29', 'End': '2018-04-13', 'Type': 'Easter'},
    {'Start': '2018-05-11', 'End': '2018-05-11', 'Type': 'Whitsun'},
    {'Start': '2018-07-09', 'End': '2018-08-18', 'Type': 'Summer'},
    {'Start': '2018-10-01', 'End': '2018-10-19', 'Type': 'Autumn'},
    {'Start': '2018-12-21', 'End': '2019-01-04', 'Type': 'Christmas'},
    
    # 2019
    {'Start': '2019-04-04', 'End': '2019-04-18', 'Type': 'Easter'},
    {'Start': '2019-05-31', 'End': '2019-05-31', 'Type': 'Whitsun'},
    {'Start': '2019-07-01', 'End': '2019-07-30', 'Type': 'Summer'}
]

# Convert to DataFrame
vacation_df = pd.DataFrame(vacation_data)
vacation_df['Start'] = pd.to_datetime(vacation_df['Start'])
vacation_df['End'] = pd.to_datetime(vacation_df['End'])

In [42]:
# Function: Is date in vacation period?
def is_in_vacation(date):
    for _, row in vacation_df.iterrows():
        if row['Start'] <= date <= row['End']:
            return 1
    return 0

# Function: Which vacation type?
def vacation_type(date):
    for _, row in vacation_df.iterrows():
        if row['Start'] <= date <= row['End']:
            return row['Type']
    return 'None'

df['Is_Vacation'] = df['date'].apply(is_in_vacation)
df['Vacation_Type'] = df['date'].apply(vacation_type)

In [43]:
print(f"Columns: {df.columns.tolist()}")
print(f"Shape: {df.shape}")
print(f"Time period: {df['date'].min()} to {df['date'].max()}")
print(f"Number of rows: {len(df):,}")
print(f"Number of product groups: {df['Warengruppe'].nunique()}")
print("\nFirst rows:")
print(df.head(10))

Columns: ['id', 'date', 'Warengruppe', 'umsatz', 'KielerWoche', 'Bewoelkung', 'Temperatur', 'Windgeschwindigkeit', 'Wettercode', 'Is_Holiday', 'Day_Before_Holiday', 'Day_After_Holiday', 'Is_Vacation', 'Vacation_Type']
Shape: (11164, 14)
Time period: 2013-07-01 00:00:00 to 2019-07-30 00:00:00
Number of rows: 11,164
Number of product groups: 6

First rows:
          id       date  Warengruppe      umsatz  KielerWoche  Bewoelkung  \
0  1307011.0 2013-07-01          1.0  148.828353          NaN         6.0   
1  1307012.0 2013-07-01          2.0  535.856285          NaN         6.0   
2  1307013.0 2013-07-01          3.0  201.198426          NaN         6.0   
3  1307014.0 2013-07-01          4.0   65.890169          NaN         6.0   
4  1307015.0 2013-07-01          5.0  317.475875          NaN         6.0   
5  1307021.0 2013-07-02          1.0  159.793757          NaN         3.0   
6  1307022.0 2013-07-02          2.0  546.780787          NaN         3.0   
7  1307023.0 2013-07-02    

# Export

In [44]:
output_file = 'complete_dataset_with_features.csv'
df.to_csv(output_file, index=False)